<a href="https://colab.research.google.com/github/Tar-ive/txst_open_datathon_2025/blob/main/dorminator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple

def initialize_system(consolidated_file: str) -> pd.DataFrame:
    """
    Load consolidated data and add tracking columns
    """
    df = pd.read_excel(consolidated_file)
    df = df.rename(columns={
        'Name': 'dorm_name',
        'Bed Capacity': 'total_beds',
        'Spaces': 'total_parking',
        'Counter': 'used_parking'  # This tracks how many spots are currently used
    })
    df['occupied_beds'] = 0  # This tracks how many beds are currently used
    return df

def check_availability(dorm_data: pd.DataFrame, dorm_name: str, needs_parking: bool) -> bool:
    """
    Check if a dorm has available space and parking if needed
    """
    try:
        dorm = dorm_data[dorm_data['dorm_name'] == dorm_name].iloc[0]
    except IndexError:
        print(f"Warning: Dorm {dorm_name} not found in data")
        return False

    # Check bed availability (95% capacity rule)
    available_beds = dorm['total_beds'] - dorm['occupied_beds']
    if (dorm['occupied_beds'] / dorm['total_beds']) >= 0.95:
        print(f"Dorm {dorm_name} is at or above 95% capacity")
        return False

    # Check parking if needed
    if needs_parking:
        available_parking = dorm['total_parking'] - dorm['used_parking']
        if available_parking <= 0:
            print(f"No parking available at {dorm_name}")
            return False

    return True

def assign_student(dorm_data: pd.DataFrame, dorm_name: str, needs_parking: bool) -> bool:
    """
    Attempt to assign a student to a dorm
    """
    if check_availability(dorm_data, dorm_name, needs_parking):
        idx = dorm_data[dorm_data['dorm_name'] == dorm_name].index[0]
        # Increment bed occupancy
        dorm_data.loc[idx, 'occupied_beds'] += 1
        # Increment parking usage if needed
        if needs_parking:
            dorm_data.loc[idx, 'used_parking'] += 1
        return True
    return False

def process_application(dorm_data: pd.DataFrame,
                       preferences: List[str],
                       needs_parking: bool) -> Tuple[str, bool]:
    """
    Process a student's application
    """
    # Try preferred dorms in order
    for dorm in preferences:
        if assign_student(dorm_data, dorm, needs_parking):
            print(f"\nAssigned to {dorm}")
            print(f"Current state of {dorm}:")
            dorm_info = dorm_data[dorm_data['dorm_name'] == dorm].iloc[0]
            print(f"Beds: {dorm_info['occupied_beds']}/{dorm_info['total_beds']}")
            print(f"Parking: {dorm_info['used_parking']}/{dorm_info['total_parking']}")
            return dorm, True

    return None, False

# Example usage
if __name__ == "__main__":
    # Load data
    dorm_data = initialize_system('ConsolidatedData.xlsx')

    # Example student applications
    students = [
        {
            'preferences': ['San Marcos', 'Butler'],
            'needs_parking': True
        },
        {
            'preferences': ['San Marcos', 'Butler'],
            'needs_parking': True
        },
        {
            'preferences': ['San Marcos'],
            'needs_parking': False
        }
    ]

    # Process applications
    for i, student in enumerate(students, 1):
        print(f"\nProcessing Student {i}:")
        result = process_application(dorm_data, student['preferences'], student['needs_parking'])
        if not result[1]:
            print("No assignment possible")

    # Display final state
    print("\nFinal State:")
    print(dorm_data[['dorm_name', 'total_beds', 'occupied_beds', 'total_parking', 'used_parking']])


Processing Student 1:

Assigned to San Marcos
Current state of San Marcos:
Beds: 1/417.0
Parking: 1.0/481

Processing Student 2:

Assigned to San Marcos
Current state of San Marcos:
Beds: 2/417.0
Parking: 2.0/481

Processing Student 3:

Assigned to San Marcos
Current state of San Marcos:
Beds: 3/417.0
Parking: 2.0/481

Final State:
              dorm_name  total_beds  occupied_beds  total_parking  \
0            San Marcos       417.0              3            481   
1                 Bexar       202.0              0            209   
2           College Inn       280.0              0             30   
3           San Jacinto       469.0              0            397   
4               Lantana       239.0              0            132   
5                Sterry       371.0              0            205   
6                Butler       236.0              0            144   
7                 Tower       434.0              0            391   
8               Jackson       423.0         